References: </br>
PKCS#1: https://datatracker.ietf.org/doc/html/rfc3447 </br>
FIPS 186.5 Digital Signature Standard : https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.186-5.pdf (for the key size) </br>
SP 900-90A for DRBG (Deterministic Random Bit Generators: https://nvlpubs.nist.gov/nistpubs/SpecialPublications/NIST.SP.800-90Ar1.pdf </br>
Public-Key Cryptography Standards (PKCS) #1: RSA Cryptography Specifications Version 2.1 https://datatracker.ietf.org/doc/html/rfc3447 </br>
NIST 800-57 part 1 </br>

Stalling's KKI book

In [96]:
import secrets
import math
import hashlib
import datetime

In [97]:
def zn_multiplication(a:int, b:int, n:int)->int:
  if a > b:
    smallest:int = b
    biggest:int = a
  else:
    smallest:int = a
    biggest:int = b
  del a, b
  str_big:str = str(bin(biggest))[2:]
  result = list()
  length = len(str_big)
  result = 0
  result += smallest * int(str_big[length-1])
  for i in range(1, length):
    smallest = (smallest << 1) % n
    result = (result + smallest*int(str_big[length-1-i]))%n
  del smallest, biggest
  return result


In [212]:
import random
for i in range(100):
    a = random.randint(200, 500)
    b = random.randint(200, 500)
    n = random.randint(90, 8000000)
    if zn_multiplication(a,b,n) != (a*b)%n:
        print(f"failed case when {a = } {b = }{n =}")

In [230]:
def zn_power(a:int, k:int, n:int)->int:
  """"Return the value of a to the power of k in xn"""
  str_k = str(bin(k))[2:][::-1]
  # print(str_k)
  result = 1
  temp = a
  for i in range(len(str_k)):
    # print(f"str_k[i] = {str_k[i]}, {temp = }")
    if(str_k[i]=='1'):
      # print(f"Multiplying {result = } by {temp = }")
      result = zn_multiplication(result, temp, n)
    temp = zn_multiplication(temp, temp, n)
  return result % n


zn_power(2, 4, 1000_000)


16

In [232]:
for i in range(100):
    a = random.randint(200, 500)
    k = random.randint(200, 500)
    n = random.randint(90, 8000000)
    if zn_power(a, k, n) != pow(a, k, n):
        print(f"failed case when {a = } {k = }{n =}")

In [228]:
def find_k_and_q(n:int)->tuple[int, int]:
  """Return a tuple (k, q). The definition is the same as the one in miler-rabin test"""
  temp = n
  s = 0
  while temp % 2 == 0:
    s += 1
    temp = temp >> 1
  return s, temp

<h1>Pseudocode of the Miller-Rabin primarality test</h1>
<img src ="./images/miller-rabin.png", width=800>
<img src ="./images/minimum-RMtest.png", width=800>

In [100]:
def miller_rabin(w:int, itterations:int)-> bool:
  """p is the prime number that will be tested, while k is the number of rounds
  the probability that p is prime is at most 4^-k.
  """
  a, m = find_k_and_q(w-1)
  b = random.randint(2, w-2)
  for i in range(itterations):
    z = zn_power(b, m, w)
    if z == 1 or z == w-1:
      continue
    for j in range(1, a):
      z = zn_multiplication(z, z, w)
      if z == w-1:
        continue
      if z == 1:
        return False #composite
  return True #probably prime


<h1>Jacobi Test Pseudocode</h1>
<img src="./images/jacobi.png" width=800>

In [101]:
def jacobi(a:int, n:int)->int:
    a = a % n
    if a ==1 or n ==1:
        return 1
    if a == 0:  
        return 0    
    e, a1 = find_k_and_q(a)
    if e%2==0:
        s = 1
    elif (n%8) == 1 or (n%8) == 7:
            s = 1
    elif (n%8) == 3 or (n%8) == 5:
        s = -1
    if ((n%4)==3 and a1 % 4 == 3):
        s = -s
    n1 = n % a1
    return jacobi(n1, a1) * s
# jacobi(5, 3439601197)

<h1>Determining Perfect Square</h1>
<img src="./images/perfect_square.png" width=800>

In [102]:
def is_perfect_square(c:int)->bool:
    """Check if n is a perfect square
    input: n - an integer
    output: True if n is a perfect square, False otherwise
    """
    n = 0
    while (1<<n) < c:
        n += 1
    m = (n//2)+1 if n%2==1 else (n//2)
    xi = secrets.randbits(m-1)+(1<<(m-1))
    while True:
        xi = (xi*xi+c)/(2*xi)
        if (xi*xi < ((1<<m)+c)):
            break
    xi = math.floor(xi)
    if c == xi*xi:
        return True #perfect square
    else:
        return False #not a perfect square
    
    

<h1>Lucas Test</h1>
<img src="./images/lucas.png" width=800>

In [103]:
def lucas_test(c:int)-> bool:
    """Lucas test for primality"""
    if is_perfect_square(c):
        return False
    s = 0
    while True:
        s+=1
        if(s%2==1):
            d = s*2+3
        else:
            d = ((s*2+3)*-1)
        jcb = jacobi(d, c)
        if jcb == 0:
            return False #composite
        if(jcb==-1 and self.gcd(c, (1-d)//4)==1):
            break
    k = c+1
    bin_k = str(bin(k))[2:]
    bin_k = bin_k[::-1] #because according to the pseudo code, it is krkr-1,...k0
    r = len(bin_k)-1 #since we start counting from 0
    u_i = 1
    v_i = 1
    inverse_2 = pow(2, -1, c)
    for i in range(r-1, -1, -1): #since the stop  is exclusive, 0 is turned into -1
        u_temp = (u_i*v_i) % c
        v_temp = ((v_i*v_i + d*u_i*u_i)*inverse_2) % c
        if bin_k[i] == '1':
            u_i = ((u_temp + v_temp)*inverse_2 )% c
            v_i = ((v_temp + d*u_temp)*inverse_2) % c
        else:
            u_i = u_temp
            v_i = v_temp
    #END FOR
    if u_i == 0:
        return True #probably prime
    else:
        return False #composite
        

    

In [104]:
def gcd(a, b):
    a = abs(a)
    b = abs(b)
    if a==0 and b==0:
        raise ValueError("GCD is undefined for 0 and 0")
    if b == 0: # Added base case for Euclidean algorithm
        return a
    while b:
        a, b = b, a % b
    return a

In [105]:
# Generate test cases with at least 5 digits
# 7 big prime numbers (all odd)
prime_numbers = [10007, 10009, 10037, 10039, 10061, 10067, 10069]

# 3 composite numbers (all odd)
composite_numbers = [10005, 10015, 10021]

# Testing lucas_test function
print("Testing Lucas Test Function:")
for prime in prime_numbers:
    result = lucas_test(prime)
    print(f"Lucas Test for prime {prime}: {result} (Expected: True)")

for composite in composite_numbers:
    result = lucas_test(composite)
    print(f"Lucas Test for composite {composite}: {result} (Expected: False)")

Testing Lucas Test Function:
Lucas Test for prime 10007: True (Expected: True)
Lucas Test for prime 10009: True (Expected: True)
Lucas Test for prime 10037: True (Expected: True)
Lucas Test for prime 10039: True (Expected: True)
Lucas Test for prime 10061: True (Expected: True)
Lucas Test for prime 10067: True (Expected: True)
Lucas Test for prime 10069: True (Expected: True)
Lucas Test for composite 10005: False (Expected: False)
Lucas Test for composite 10015: False (Expected: False)
Lucas Test for composite 10021: False (Expected: False)


<h1>Determining Perfect Square</h1>
<img src="./images/perfect_square.png" width=800>

<h1>DRBH HASH</h1>
<img src="./images/hash_drbg.png" width=800>

In [1]:
def long_to_bytes(n: int) -> bytes:
    """Convert a long integer to bytes."""
    length = (n.bit_length() + 7) // 8 or 1
    return n.to_bytes(length)


def bytes_to_long(b: bytes) -> int:
    """Convert bytes to a long integer."""
    return int.from_bytes(b)
long_to_bytes(1)

b'\x01'

In [4]:
import secrets
import datetime
import hashlib
import math
import gmpy2
class HashDRBG():
    def __init__(self, seedlen:int):
        self.seedlen = seedlen
        self.personalization_string = b'NeverGonnaGiveYouUp'
        self.C = None
        self.V = None
        self.reseed_counter = 1
        self.reseed_interval = 5
        self.seed_material = None
        self.seed = None
        self.outlen = 256

        self.__initialize_state()  # Initial reseed to set up the DRBG
    
    def __generate_nonce(self)-> bytes:
        """Generate a nonce for the DRBG"""
        temp = int(datetime.datetime.now(datetime.timezone.utc).timestamp()*1000000)
        return temp.to_bytes(length=(temp.bit_length() // 8) + 1) 
    def __Hash_df(self, m:bytes)-> bytes:
        """Return the result of hashing m with SHA-256. The outleng is 256 bits"""
        return hashlib.sha256(m).digest()
    def __initialize_state(self):
        """Reseed the DRBG with new entropy"""
        entropy_input = secrets.token_bytes(self.seedlen // 8)
        nonce = self.__generate_nonce()
        self.seed_material = entropy_input + nonce + self.personalization_string
        self.seed = self.__Hash_df(self.seed_material)
        self.V = self.seed
        self.C = self.__Hash_df(b'00000000'+ self.V)
        self.reseed_counter = 1

    def __reseed(self, additional_input:bytes = b''):
        """Reseed the DRBG with additional input"""
        entropy_input = secrets.token_bytes(self.seedlen // 8)
        self.seed_material = b"00000001" +self.V+ entropy_input + additional_input
        self.seed = self.__Hash_df(self.seed_material)
        self.V = self.seed
        self.C = self.__Hash_df(b'00000001' + self.V)
        self.reseed_counter = 1
    def leftmost_bits(self, data: bytes, n: int) -> bytes:
        """
        Return the n leftmost bits of 'data', as a bytes object of length ceil(n/8).

        - data: input bytes (big‑endian bit order).
        - n: number of bits to extract (must be >= 0).
        - If n == 0: returns b''.
        - If n <= len(data)*8: takes the top n bits, dropping the rest.
        - If n > len(data)*8: takes all bits of data, then pads with (n - len(data)*8) zeros.
        - The returned bytes are big‑endian; any unused low‑order bits in the last byte are zeros.
        """
        if n < 0:
            raise ValueError("n must be non-negative")
        if n == 0:
            return b''

        total_bits = len(data) * 8
        x = int.from_bytes(data, 'big')
        if n > total_bits:
            raise ValueError(f"n ({n}) is greater than the total bit width of data ({total_bits})")
        # drop (total_bits - n) rightmost bits
        x >>= (total_bits - n)


        # figure out how many bytes we need
        out_len = (n + 7) // 8
        return x.to_bytes(out_len, 'big')

    def __hash_gen(self, requested_bits:int) -> bytes:
        """Generate hash output based on the current state"""
        output = b''
        m = math.ceil(requested_bits / self.outlen)
        data = bytes_to_long(self.V)
        for i in range(m):
            # print(i)
            w = self.__Hash_df(long_to_bytes(data))
            output = output + w
            data = (data + 1) % 2**self.seedlen
        return self.leftmost_bits(output, requested_bits)
    def generate_ramdom_bits(self, requested_bits:int) -> bytes:
        """Generate random bytes using the DRBG"""
        if self.reseed_counter >= self.reseed_interval:
            self.__reseed()
        self.reseed_counter += 1
        output = self.__hash_gen(requested_bits)
        H = self.__Hash_df(b"00000003"+ self.V)
        self.V = long_to_bytes(bytes_to_long(self.V + H+long_to_bytes(self.reseed_counter)) % 2**self.seedlen)
        return output
    def generate_random_int(self, min_value:int, max_value:int) -> int:
        """Generate a random integer in the range [min_value, max_value)"""
        if min_value >= max_value:
            raise ValueError("min_value must be less than max_value")
        range_size = max_value - min_value
        if range_size <= 0:
            raise ValueError("Range size must be greater than 0")
        bit_size:int = int(gmpy2.ceil(gmpy2.log2(range_size+1)))
        while True:
            random_bytes = self.generate_ramdom_bits(bit_size)
            random_int = bytes_to_long(random_bytes)
            if random_int < range_size:
                return min_value + random_int


b'\xfe\x0e\xc8\x82N9\x7f\xb9\xd5\xe6\xaeF\xf8\xcd\xbd\xab6\x90\xcf5\xe6\x92S?\xd8\xb3\x90\x9a\t}\x8byX\xd9\x00\xdb+\xb9\x10,\xc7\xa0\xa9\x92\xdc\xfa:a\x15\x01B\xd0w}\x07\x18\xdbwF\xf6"O\xef\xa7\xde\xfb&q5\x81\xfd\xb3\xfd\x8a\xfa\xeb\xdd\xfa\t\x9c\x1e\x00==\x1c\x90\x9c\xd5\x8dr[;\xb5\xa6\xadM\x93C\xb9\x915\x98\x98\x00\x9c\xad\x91\xfe\xb2\x9b\xdc%y\x00vk$\xfd\xec\x94z\xe0\x10,\xac\xaa\xad\xdc\xa5\xc7G\xf6\xb5\x86\x97\x06\x11\xf6"\xca\x88\x80:t3,\xa3jQ\xca\xa3\xbd\xf0\xce\xcf0\xca\xec\xecG\xf2,~\x8a\n\x1bq\x9d\xf0gv\xb9/\xfeo\x91\x9d\x8fr\xd9\x12\x8c\xe1\x19\x14\xec \x8c"\xbbcfo\x83a\xf5w\x99n\x04\x92\x04\xf4\xe5\x19\xf0]\x9a\x8e[1\xf9\xb9\xb8\x15\xc17+l\x1e\xc2\xad\x90\xb0z\xbcF\xad`"`^\xea!\\\xaf\x7f\xe8e\xc8\xbd\xff\'\xf4\xf9K \xc4\xf4\xc5\x11\xcf\x12RbeR\xd2z\xb1\x1c\x0f\xa4\xc0\x7f\xfc\xc9/\xcaJ\x0ca\x97\xb4\x10s\x13\xe6\xec\xd5\x08%p\x05\xb0\xa7\xb6\x18\xe3T}\xe4\x89\xdb`Y\x8a\xefns\xaej\x06\xf5hD\xebU\x9c\xfd\xf4\x05t\xec\xf4\xc4\x96=\xc5\xf2\xd6\xbf\x98\xfa\xc6\x8eD\xd8\xcd\xac\xa

In [73]:
import json
class RSA():
    def __init__(self):
        self.p = None
        self.q = None
        self.n = None
        self.e = None
        self.d = None
        self.drbg = HashDRBG(seedlen=256)  
        self.security_strength = 128
        self.nlen = 3072 #This is hardcoded in respect to SP800-57, Part 1 for security_strength 128
        self.min_mr = 4
    def __long_to_bytes(self,n: int) -> bytes:
        """Convert a long integer to bytes."""
        length = (n.bit_length() + 7) // 8 or 1
        return n.to_bytes(length)
    def __zn_multiplication(self, a:int, b:int, n:int)->int:
        if a > b:
            smallest:int = b
            biggest:int = a
        else:
            smallest:int = a
            biggest:int = b
        del a, b
        str_big:str = str(bin(biggest))[2:]
        result = list()
        length = len(str_big)
        result = 0
        result += smallest * int(str_big[length-1])
        for i in range(1, length):
            smallest = (smallest << 1) % n
            result = (result + smallest*int(str_big[length-1-i]))%n
        del smallest, biggest
        return result

    def __zn_power(self, a:int, k:int, n:int)->int:
      """"Return the value of a to the power of k in xn"""
      str_k = str(bin(k))[2:][::-1]
      # print(str_k)
      result = 1
      temp = a
      for i in range(len(str_k)):
        # print(f"str_k[i] = {str_k[i]}, {temp = }")
        if(str_k[i]=='1'):
          # print(f"Multiplying {result = } by {temp = }")
          result = self.__zn_multiplication(result, temp, n)
        temp = self.__zn_multiplication(temp, temp, n)
      return result % n

    def __bytes_to_long(self, b: bytes) -> int:
        """Convert bytes to a long integer."""
        return int.from_bytes(b)
    def __gcd(self, a, b):
        a = abs(a)
        b = abs(b)
        if a==0 and b==0:
            raise ValueError("GCD is undefined for 0 and 0")
        if b == 0: # Added base case for Euclidean algorithm
            return a
        while b:
            a, b = b, a % b
        return a
    def __is_perfect_square(self, c:int)->bool:
        """Check if n is a perfect square
        input: n - an integer
        output: True if n is a perfect square, False otherwise
        """
        n = 0
        while (1<<n) < c:
            n += 1
        m = (n//2)+1 if n%2==1 else (n//2)
        xi = gmpy2.mpq(self.drbg.generate_random_int(2**(m-1), 2**m)) #wrapping it with gmpy2.mpz to avoid float conversion errors
        while True:
            xi = (xi*xi+c)/(2*xi)
            if (xi*xi < ((1<<m)+c)):
                break
        xi = math.floor(xi)
        if c == xi*xi:
            return True #perfect square
        else:
            return False #not a perfect square
    
    
    def __find_k_and_q(self, n:int)->tuple[int, int]:
        """Return a tuple (k, q). The definition is the same as the one in miler-rabin test"""
        temp = int(n)
        s = 0
        while temp % 2 == 0:
            s += 1
            temp = temp >> 1
        return s, temp
    def __jacobi(self, a:int, n:int)->int:
        a = a % n
        if a ==1 or n ==1:
            return 1
        if a == 0:  
            return 0    
        e, a1 = self.__find_k_and_q(a)
        if e%2==0:
            s = 1
        elif (n%8) == 1 or (n%8) == 7:
                s = 1
        elif (n%8) == 3 or (n%8) == 5:
            s = -1
        if ((n%4)==3 and a1 % 4 == 3):
            s = -s
        n1 = n % a1
        return jacobi(n1, a1) * s
# jacobi(5, 3439601197)
    def __miller_rabin(self, w:int, k:int)-> bool:
        """p is the prime number that will be tested, while k is the number of rounds
        input: 
        w - the number to be tested, itterations - the number of rounds.
        k - the number of rounds
        """
        a, m = self.__find_k_and_q(w-1)
        for i in range(k):
            b = (self.drbg.generate_random_int(2, w-2))
            if not (1<b<w-1):
                continue
            z = zn_power(b, m, w)
            if (z==1 or z==w-1):
                continue
            for j in range(1, a):
                z = zn_multiplication(z, z, w)
                if z == w-1:
                    break
                if z == 1:
                    return False
            else:
                return False
        return True #probably prime
    def __lucas_test(self, c:int)-> bool:
        """Lucas test for primality"""
        if self.__is_perfect_square(c):
            return False
        s = 0
        while True:
            print(s)
            s+=1
            if(s%2==1):
                d = s*2+3
            else:
                d = ((s*2+3)*-1)
            jcb = gmpy2.jacobi(d, c)
            if jcb == 0:
                return False #composite
            if(jcb==-1 and self.__gcd(c, (1-d)//4)==1):
                break
        k = c+1
        bin_k = str(bin(k))[2:]
        bin_k = bin_k[::-1] #because according to the pseudo code, it is krkr-1,...k0
        r = len(bin_k)-1 #since we start counting from 0
        u_i = 1
        v_i = 1
        inverse_2 = pow(2, -1, c)
        for i in range(r-1, -1, -1): #since the stop  is exclusive, 0 is turned into -1
            u_temp = (u_i*v_i) % c
            v_temp = ((v_i*v_i + d*u_i*u_i)*inverse_2) % c
            if bin_k[i] == '1':
                u_i = ((u_temp + v_temp)*inverse_2 )% c
                v_i = ((v_temp + d*u_temp)*inverse_2) % c
            else:
                u_i = u_temp
                v_i = v_temp
        #END FOR
        if u_i == 0:
            return True #probably prime
        else:
            return False #composite
    def lucas_test(self, c:int)-> bool:
        """Lucas test for primality"""
        if self.__is_perfect_square(c):
            return False
        s = 0
        while True:
            print(s)
            s+=1
            if(s%2==1):
                d = s*2+3
            else:
                d = ((s*2+3)*-1)
            jcb = gmpy2.jacobi(d, c)
            if jcb == 0 and abs(c) != abs(d):
                return False #composite
            if(jcb==-1 and self.__gcd(c, (1-d)//4)==1):
                break
        k = c+1
        bin_k = str(bin(k))[2:][::-1]
        r = len(bin_k)-1 #since we start counting from 0
        u_i = 1
        v_i = 1
        inverse_2 = pow(2, -1, c)
        for i in range(r-1, -1, -1): #since the stop  is exclusive, 0 is turned into -1
            u_temp = (u_i*v_i) % c
            v_temp = ((v_i*v_i + d*u_i*u_i)*inverse_2) % c
            if bin_k[i] == '1':
                u_i = ((u_temp + v_temp)*inverse_2 )% c
                v_i = ((v_temp + d*u_temp)*inverse_2) % c
            else:
                u_i = u_temp
                v_i = v_temp
        #END FOR
        if u_i == 0:
            return True #probably prime
        else:
            return False #composite

    def __get_probable_prime(self, e:int, a:int=None, b:int = None) -> int: #there is a mechanism that generates provable primes, but we opt for probable primes
        """Generate a probable prime number with the given security strength
        Input: 
        e - public exponent
        a,b - elements of {1,3,5,7} if one wants to specify p % 8 == a or p % 8 ==1. 
        """
        if self.nlen < 2048:
            raise ValueError("nlen must be at least 2048 bits")
        if not ((16<math.log2(e)<256) or e % 2 == 1): #checking if e follows the constrains
            raise ValueError("e must be an odd integer between 16 and 256 bits")
        # Generate p
        i = 0 
        while True:
            print(i)
            ub = 2**(self.nlen//2)
            lb = (((2**(self.nlen//2-1)) * int(math.sqrt(2)*1e12)) //int(1e12))
            p = (self.drbg.generate_random_int(lb, ub))
            if a is not None:
                p = p + ((a-p)%8)
            if p % 2 == 0:
               p +=1 
            if p < (((2**(self.nlen//2-1)) * int(math.sqrt(2)*1e12)) //int(1e12)): #to avoid float conversion error 
               continue
            if self.__gcd(p-1, e) == 1:
                print('a')
            #    if self.__miller_rabin(p, self.min_mr*2):
                if gmpy2.mpz(p).is_probab_prime(self.min_mr*2):
                    print('b')
                    if self.__lucas_test(p):
                        self.p = p
                        print('c')
                        break
            i += 1
            if i > self.nlen*5:
                raise Exception("Failed to generate a probable prime after many attempts")
        # Generate q
        print('generating q')
        i = 0
        while True:
            print(f"{i = }")
            q  =bytes_to_long(self.drbg.generate_ramdom_bits(self.nlen//2))
            if b is not None:
                q = q + ((b-q)%8)
            if q % 2 == 0:
               q +=1 
            print('e')
            if q < (((2**(self.nlen//2-1)) * int(math.sqrt(2)*1e12)) //int(1e12)):
                continue
            print('f')
            if (abs(p-q)<((2**(self.nlen//2-100)))):
                continue
            print('g')
            if self.__gcd(q-1, e) == 1:
                print('d')
                if gmpy2.mpz(q).is_probab_prime(self.min_mr*2):
                    if self.__lucas_test(q):
                        print('e')
                        self.q = q
                        break
            i += 1
            if i > self.nlen*10:
                raise Exception("Failed to generate a probable prime after many attempts")
    def __extended_euclidian_algorithm(self, a, b):

        # Base case for recursive extended Euclidean algorithm
        if a == 0:
            # gcd(0, b) = b.  The equation is 0*x + b*y = b.  So x=0, y=1.
            return b, 0, 1 

        # Recursive call: modular_inverse(b % a, a)
        # This finds gcd(b % a, a) and coefficients x', y' such that (b % a)x' + ay' = gcd(b % a, a)
        gcd_val, x1_rec, y1_rec = self.__extended_euclidian_algorithm(b % a, a)

        x_rec = y1_rec - (b // a) * x1_rec  # this is x for original 'a' (which is current 'b' in recursive call)
        y_rec = x1_rec                      # this is y for original 'b' (which is current 'a' in recursive call)
                                            # the roles of a and b are swapped in the recursive call's perspective
                                            # relative to the formula ax + by = gcd(a,b)

        return gcd_val, x_rec, y_rec
    def __modular_inverse(self, a, m):
        gcd_val, x, y = self.__extended_euclidian_algorithm(a, m)
        if gcd_val != 1:
            raise ValueError(f"No modular inverse exists for {a} mod {m}")
        return x % m  # x might be negative, so we take it modulo m to get a positive result
    def innitialize_rsa(self, e:int, a:int=None, b:int=None):
        """Initialize RSA with the given public exponent and optional constraints for p and q"""
        if self.p is not None or self.q is not None:
            raise Exception("RSA is already initialized")
        self.__get_probable_prime(e, a, b)
        self.n = self.p * self.q
        phi = (self.p-1)*(self.q-1)
        if self.__gcd(e, phi) != 1:
            raise ValueError("e must be coprime to phi(n)")
        self.e = e
        self.d = self.__modular_inverse(self.e, phi)
    def encrypt(self, plaintext: bytes) -> bytes:
        """Encrypt the plaintext using RSA public key"""
        if self.n is None or self.e is None:
            raise Exception("RSA is not initialized")
        plaintext_int = self.__bytes_to_long(plaintext)
        if plaintext_int >= self.n:
            raise ValueError("Plaintext must be less than n")
        ciphertext_int = self.__zn_power(plaintext_int, self.e, self.n)
        ciphertext = self.__long_to_bytes(ciphertext_int)
        return ciphertext
    def decrypt(self, ciphertext: bytes) -> bytes:
        """Decrypt the ciphertext using RSA private key"""
        if self.n is None or self.d is None:
            raise Exception("RSA is not initialized")
        ciphertext_int = self.__bytes_to_long(ciphertext)
        plaintext_int = self.__zn_power(ciphertext_int, self.d, self.n)
        plaintext = self.__long_to_bytes(plaintext_int)
        return plaintext
    def get_public_key(self) -> tuple[int, int]:
        """Return the public key (n, e)"""
        if self.n is None or self.e is None:
            raise Exception("RSA is not initialized")
        return self.n, self.e    
    def get_private_key(self) -> tuple[int, int]:
        """Return the private key (n, d)"""
        if self.n is None or self.d is None:
            raise Exception("RSA is not initialized")
        return self.n, self.d
    def save_state(self, filename: str):
        """Save the RSA state to a .json file"""
        state = {
            'p': self.__long_to_bytes(self.p),
            'q': self.__long_to_bytes(self.q),
            'n': self.__long_to_bytes(self.n),
            'e': self.e,
            'd': self.__long_to_bytes(self.d)
        }
        with open(filename, 'w') as f:
            json.dump(state, f)
    def load_state(self, filename: str):
        """Load the RSA state from a file"""
        with open(filename, 'rb') as f:
            state = json.load(f)
            self.p = self.__bytes_to_long(state['p'])
            self.q = self.__bytes_to_long(state['q'])
            self.n = self.__bytes_to_long(state['n'])
            self.e = state['e']
            self.d = self.__bytes_to_long(state['d'])
            if not (self.p and self.q and self.n and self.e and self.d):
                raise ValueError("Invalid RSA state in the file")

In [40]:
def miller_rabin(w:int, k:int)-> bool:
    """p is the prime number that will be tested, while k is the number of rounds
    input: 
    w - the number to be tested, itterations - the number of rounds.
    k - the number of rounds
    """
    a, m = find_k_and_q(w-1)
    wlen = int(gmpy2.ceil(gmpy2.log2(w+1)))
    for i in range(k):
        b = (secrets.randbits(wlen))
        if not (1<b<w-1):
            continue
        z = zn_power(b, m, w)
        if (z==1 or z==w-1):
            continue
        for j in range(1, a):
            z = zn_multiplication(z, z, w)
            if z == w-1:
                break
            if z == 1:
                return False
        else:
            return False
    return True #probably prime
        


In [64]:
rsa = RSA()
rsa.innitialize_rsa(e=65537, a=3, b=5)

0
a
1
a
2
a
3
a
4
a
5
a
6
a
7
a
8
a
9
a
10
a
11
a
12
a
13
a
14
a
15
a
16
a
17
a
18
a
19
a
20
a
21
a
22
a
23
a
24
a
25
a
26
a
27
a
28
a
29
a
30
a
31
a
32
a
33
a
34
a
35
a
36
a
37
a
38
a
39
a
40
a
41
a
42
a
43
a
44
a
45
a
46
a
47
a
48
a
49
a
50
a
51
a
52
a
53
a
54
a
55
a
56
a
57
a
58
a
59
a
60
a
61
a
62
a
63
a
64
a
65
a
66
a
67
a
68
a
69
a
70
a
71
a
72
a
73
a
74
a
75
a
76
a
77
a
78
a
79
a
80
a
81
a
82
a
83
a
84
a
85
a
86
a
87
a
88
a
89
a
90
a
91
a
92
a
93
a
94
a
95
a
96
a
97
a
98
a
99
a
100
a
101
a
102
a
103
a
104
a
105
a
106
a
107
a
108
a
109
a
110
a
111
a
112
a
113
a
114
a
115
a
116
a
117
a
118
a
119
a
120
a
121
a
122
a
123
a
124
a
125
a
126
a
127
a
128
a
129
a
130
a
131
a
132
a
133
a
134
a
135
a
136
a
137
a
138
a
139
a
140
a
141
a
142
a
143
a
144
a
145
a
146
a
147
a
148
a
149
a
150
a
151
a
152
a
153
a
154
a
155
a
156
a
157
a
158
a
159
a
160
a
161
a
162
a
163
a
164
a
165
a
166
a
167
a
168
a
169
a
170
a
171
a
172
a
173
a
174
a
175
a
176
a
177
a
178
a
179
a
180
a
181
a
182
a
183
a
184
a


2

In [68]:
rsa.q

2405451138665545567673681958144108477293375623356069468897818496953955962408985733885481045623424364297983824181298631173171042573646756520786150782191279356438381021195705402937353417575589812164071094466955234633267210444805550680191817003656997332971411529196903373259591406230616110759091434044827605062586131336148148702047207660683532792080844789917247884753863207633083743382967132371220981019314521089091150931821409985892789676618119631743859390530163901

In [33]:
import pytest

def test_lucas_test():
    # Positive test cases (prime numbers)
    assert rsa.lucas_test(3) == True  # Small prime
    assert rsa.lucas_test(5) == True  # Small prime
    assert rsa.lucas_test(7) == True  # Small prime
    assert rsa.lucas_test(11) == True  # Small prime
    assert rsa.lucas_test(13) == True  # Small prime
    assert rsa.lucas_test(17) == True  # Small prime
    assert rsa.lucas_test(19) == True  # Small prime
    assert rsa.lucas_test(23) == True  # Small prime
    assert rsa.lucas_test(29) == True  # Small prime
    assert rsa.lucas_test(101) == True  # Medium prime
    assert rsa.lucas_test(103) == True  # Medium prime
    assert rsa.lucas_test(107) == True  # Medium prime
    assert rsa.lucas_test(109) == True  # Medium prime
    assert rsa.lucas_test(113) == True  # Medium prime
    assert rsa.lucas_test(1009) == True  # Medium prime
    assert rsa.lucas_test(7919) == True  # Large prime
    assert rsa.lucas_test(999983) == True  # Large prime
    # assert rsa.lucas_test(10000000000000037) == True  # Very large prime
    assert rsa.lucas_test(10000000000000061) == True  # Very large prime

    # Negative test cases (composite numbers)
    assert rsa.lucas_test(1) == False  # Not prime
    assert rsa.lucas_test(9) == False  # Small composite
    assert rsa.lucas_test(15) == False  # Small composite
    assert rsa.lucas_test(16) == False  # Small composite
    assert rsa.lucas_test(100) == False  # Medium composite
    assert rsa.lucas_test(121) == False  # Medium composite
    assert rsa.lucas_test(1001) == False  # Medium composite
    assert rsa.lucas_test(7957) == False  # Large composite
    assert rsa.lucas_test(999981) == False  # Large composite
    assert rsa.lucas_test(10000000000000000) == False  # Very large composite
    assert rsa.lucas_test(10000000000000033) == False  # Very large composite
    assert rsa.lucas_test(10000000000000057) == False  # Very large composite
test_lucas_test()

0
0
1
0
0
1
2
3
4
0
0
0
1
0
0
1
2
3
0
1
0
0
0
1
2
3
0
0
1
2
3
0
1
2
3
0
0
1
2
3
0
0
1
0
0
1
0
0


In [296]:
rsa.lucas_test(5)

0


False

In [289]:
rsa.lucas_test(1000003)

0


True

In [68]:
rsa.lucas_test(temp)

0
1
2
3
4


False

In [29]:
rsa = RSA()
rsa.innitialize_rsa(e=65537, a=3, b=5)


0
0
a
b
s =  1
1
a
b
s =  1
s =  2
2
2
a


KeyboardInterrupt: 

In [31]:
def get_leftmost_bits(x: int, n: int, total_bits: int = None) -> bytes:
    """
    Return the n leftmost bits of x, as an integer.
    
    :param x: the input number (treated as a bit‐string)
    :param n: how many bits from the left to extract
    :param total_bits: (optional) the width of x in bits; 
                       if omitted, uses x.bit_length()
    :return: bits that are is the leftmost n bits of x
    """
    if x < 0:
        raise ValueError("x must be non-negative")
    width = total_bits if total_bits is not None else x.bit_length()
    if n > width:
        raise ValueError(f"n ({n}) is greater than the total bit width ({width})")
    shift = width - n
    mask  = (1 << n) - 1
    return bin((x >> shift) & mask)


In [33]:
(get_leftmost_bits(0b1101010101,5))  # Should return 0b1101 (13 in decimal)

'0b11010'

In [44]:
def extended_euclidian_algorithm(a, b):

    # Base case for recursive extended Euclidean algorithm
    if a == 0:
        # gcd(0, b) = b.  The equation is 0*x + b*y = b.  So x=0, y=1.
        return b, 0, 1 

    # Recursive call: modular_inverse(b % a, a)
    # This finds gcd(b % a, a) and coefficients x', y' such that (b % a)x' + ay' = gcd(b % a, a)
    gcd_val, x1_rec, y1_rec = extended_euclidian_algorithm(b % a, a)

    x_rec = y1_rec - (b // a) * x1_rec  # this is x for original 'a' (which is current 'b' in recursive call)
    y_rec = x1_rec                      # this is y for original 'b' (which is current 'a' in recursive call)
                                        # the roles of a and b are swapped in the recursive call's perspective
                                        # relative to the formula ax + by = gcd(a,b)

    return gcd_val, x_rec, y_rec
def modular_inverse(a, m):
    gcd_val, x, y = extended_euclidian_algorithm(a, m)
    if gcd_val != 1:
        raise ValueError(f"No modular inverse exists for {a} mod {m}")
    return x % m  # x might be negative, so we take it modulo m to get a positive result

1